# **Исследование надёжности заёмщиков**

**Описание проекта**


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.


Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Описание данных:
- **children** — количество детей в семье
- **days_employed** — общий трудовой стаж в днях
- **dob_years** — возраст клиента в годах
- **education** — уровень образования клиента
- **education_id** — идентификатор уровня образования
- **family_status** — семейное положение
- **family_status_id** — идентификатор семейного положения
- **gender** — пол клиента
- **income_type** — тип занятости
- **debt** — имел ли задолженность по возврату кредитов
- **total_income** — ежемесячный доход
- **purpose** — цель получения кредита

# Подключение библиотек

In [1]:
import numpy as np
import pandas as pd

# Шаг 1. Общая информация и замечания по данным.

## 1.1. Изучение общей информации.

In [2]:
df = pd.read_csv("datasets/data.csv")

In [3]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Количество NA значений по столбцам.

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах **days_employed** и **total_income** одинаковое количество NA значений. Возможно в одних и тех же строках, надо будет проверить.

In [6]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


## 1.2. Ближе изучаем данные, находим замечания.

### 1.2.1.

In [7]:
df["children"].value_counts().sort_index()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: count, dtype: int64

В столбце "children" присутствуют странные значения -1 и 20. 20 детей очень большая редкость, 76 значений слишком много для реального случая. Это не единичные случаи, возможно это ошибка при занесении данных в таблицу или выгрузке. Исправим -1 на 0, а в числе 20 уберём лишний ноль. 

Наша гипотеза потвердилась, NA значения в этих двух столбцах в одних и тех же строках.

### 1.2.2. **days_employed**.

In [8]:
df[(df["days_employed"].isna() == True) & (df["total_income"].isna() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


In [21]:
(df[df["days_employed"] < 0].shape[0]) / (len(df["days_employed"]))

0.7389547038327526

74% данных в столбце **days_employed** это отрицательные значения. Это не выбросы, это ошибка в данных, скорее всего при автоматической записи данных, отнимались отработанные дни из общего числа дней даты увольнения.

In [27]:
df["income_type"].value_counts()

income_type
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: count, dtype: int64

In [24]:
df[df["days_employed"] < 0]["income_type"].value_counts()

income_type
сотрудник          10014
компаньон           4577
госслужащий         1312
студент                1
предприниматель        1
в декрете              1
Name: count, dtype: int64

В эти отрицательные значения не входят пенсионеры и безработные. Можно взять значения по модулю.

In [12]:
df.groupby("income_type")["days_employed"].describe()

,count,mean,std,min,25%,50%,75%,max
income_type,,,,,,,,
безработный,2.0,366413.652744,40855.478519,337524.466835,351969.059790,366413.652744,380858.245699,395302.838654
в декрете,1.0,-3296.759962,NaN,-3296.759962,-3296.759962,-3296.759962,-3296.759962,-3296.759962
госслужащий,1312.0,-3399.896902,2788.371363,-15193.032201,-4759.399260,-2689.368353,-1257.171811,-39.954170
компаньон,4577.0,-2111.524398,2048.448594,-17615.563266,-2876.648520,-1547.382223,-685.687432,-30.195337
пенсионер,3443.0,365003.491245,21069.606065,328728.720605,346649.346146,365213.306266,383231.396871,401755.400475
предприниматель,1.0,-520.848083,NaN,-520.848083,-520.848083,-520.848083,-520.848083,-520.848083
сотрудник,10014.0,-2326.499216,2307.924129,-18388.949901,-3108.123025,-1574.202821,-746.027361,-24.141633
студент,1.0,-578.751554,NaN,-578.751554,-578.751554,-578.751554,-578.751554,-578.751554


У пенсионеров и безработных стаж работы варируется от 900 до 1100 лет. Они в своих группах, и не будут воздействовать на других работников. Да и наше исследование не предполагает изучение влияния стажа. Мы будем исходить только из столбца **income_type**.

Находим распределение NA значений по типу занятости.

In [42]:
df[df["days_employed"].isna() == True]["income_type"].value_counts()

income_type
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: count, dtype: int64

NA значения распределены по всем типам занятости, значит будем брать среднее для каждой группы чтобы заполнить NA.

## Вывод (Что надо исправить):
| <span style="color:orange">1. Странные значения</span> | <span style="color:orange">2. Пропуски</span> | <span style="color:orange">3. Замена типа данных</span> | <span style="color:orange">4. Удаление дубликатов</span> | <span style="color:orange">5. Лемматизация</span> | <span style="color:orange">6. Категоризация</span> |
| - | - | - | - | - | - |
| **children**: Поменять -1 на 0, а 20 на 2. | **days_employed**: вычислить среднее для каждой группы по типу занятости и заполнить NA. |
| **days_employed**: поменять на значения по модулю. |